# Title of Model/Problem

#Sea wharf advertising budget allocation


Install amplpy and other packages

In [36]:
# Install dependencies
!pip install -q amplpy ampltools

In [37]:
# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


Define model.

In [38]:
ampl.eval ('''
reset;
set M;
param medium_value{i in M};
param total_budget;
param min_pct;
param min_ratio_news_to_radio;
var x{i in M} >= 0;

maximize total_value_objective: sum{i in M} medium_value[i]*x[i];
subject to budget_constraint: sum{i in M} x[i]<=total_budget;
subject to min_pct_constraint {i in M}: x[i] >= min_pct*total_budget;
subject to min_ratio_news_to_radio_constraint: x['newspaper']/x['radio'] >= min_ratio_news_to_radio;

''')

In [39]:
medium_value={"radio":80,"newspaper":50}

In [40]:
M=["radio", "newspaper"]

In [41]:
print(medium_value)

{'radio': 80, 'newspaper': 50}


Define data and provide to AMPL model

In [42]:

ampl.set['M'] = M
ampl.param['medium_value'] = medium_value
ampl.param['min_pct'] = 0.25
ampl.param['total_budget'] = 1000
ampl.param['min_ratio_news_to_radio'] = 2.0
...

Ellipsis

Use ampl.expand to confirm AMPL model syntax is working

In [43]:
ampl.eval('''expand;''')

maximize total_value_objective:
	80*x['radio'] + 50*x['newspaper'];

subject to budget_constraint:
	x['radio'] + x['newspaper'] <= 1000;

subject to min_pct_constraint['radio']:
	x['radio'] >= 250;

subject to min_pct_constraint['newspaper']:
	x['newspaper'] >= 250;

subject to min_ratio_news_to_radio_constraint
	:
	x['newspaper']/x['radio'] >= 2;



Set solver option and solve

In [44]:
ampl.setOption('solver', 'cbc')
ampl.solve()

cbc 2.10.10: Cbc3007W No integer variables - nothing to do

------------ WARNINGS ------------
WARNING.  3 case(s) of "PLApprox". One of them:
  An expression of type 'Pow' has been
piecewise-linearly approximated. Set cvt:plapprox:reltol
to control precision (currently 0.010000).
WARNING.  3 case(s) of "PLApproxDomain". One of them:
  Argument domain of a 'Pow'
has been reduced to [-316.227766, 316.227766] for numerical reasons
(partially controlled by cvt:plapprox:domain.)
cbc 2.10.10: optimal solution; objective 59420.02421
0 simplex iterations
 
------------ WARNINGS ------------
WARNING.  3 case(s) of "PLApprox". One of them:
  An expression of type 'Pow' has been
piecewise-linearly approximated. Set cvt:plapprox:reltol
to control precision (currently 0.010000).
WARNING.  3 case(s) of "PLApproxDomain". One of them:
  Argument domain of a 'Pow'
has been reduced to [-316.227766, 316.227766] for numerical reasons
(partially controlled by cvt:plapprox:domain.)
  Type                  

Get and print results

In [45]:
# Print results
obj = ampl.get_objective('total_value_objective')
print("\n", "Total exposure is: ", obj.get().value(), "\n")
print("allocation:")
ampl.display('x');


 Total exposure is:  59420.024209424526 

allocation:
x [*] :=
newspaper  685.999
    radio  314.001
;



The optimal solution is to allocate $666.67 to newspaper advertising and $333.33 to radio , for a total $1000 of ...